In [2]:
#Predictive model analyzing relationship between race and employment status, employer satisfaction
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
#reading data
data = pd.read_csv('HRDataset_v14.csv')
pd.set_option('display.max_columns', None)
data

,Employee_Name,EmpID,MarriedID,MaritalStatusID,GenderID,EmpStatusID,DeptID,PerfScoreID,FromDiversityJobFairID,Salary,Termd,PositionID,Position,State,Zip,DOB,Sex,MaritalDesc,CitizenDesc,HispanicLatino,RaceDesc,DateofHire,DateofTermination,TermReason,EmploymentStatus,Department,ManagerName,ManagerID,RecruitmentSource,PerformanceScore,EngagementSurvey,EmpSatisfaction,SpecialProjectsCount,LastPerformanceReview_Date,DaysLateLast30,Absences
0,"Adinolfi, Wilson K",10026,0,0,1,1,5,4,0,62506,0,19,Production Technician I,MA,1960,7/10/1983,M,Single,US Citizen,No,White,7/5/2011,NaN,N/A-StillEmployed,Active,Production,Michael Albert,22.0,LinkedIn,Exceeds,4.60,5,0,1/17/2019,0,1
1,"Ait Sidi, Karthikeyan",10084,1,1,1,5,3,3,0,104437,1,27,Sr. DBA,MA,2148,5/5/1975,M,Married,US Citizen,No,White,3/30/2015,6/16/2016,career change,Voluntarily Terminated,IT/IS,Simon Roup,4.0,Indeed,Fully Meets,4.96,3,6,2/24/2016,0,17
2,"Akinkuolie, Sarah",10196,1,1,0,5,5,3,0,64955,1,20,Production Technician II,MA,1810,9/19/1988,F,Married,US Citizen,No,White,7/5/2011,9/24/2012,hours,Voluntarily Terminated,Production,Kissy Sullivan,20.0,LinkedIn,Fully Meets,3.02,3,0,5/15/2012,0,3
3,"Alagbe,Trina",10088,1,1,0,1,5,3,0,64991,0,19,Production Technician I,MA,1886,9/27/1988,F,Married,US Citizen,No,White,1/7/2008,NaN,N/A-StillEmployed,Active,Production,Elijiah Gray,16.0,Indeed,Fully Meets,4.84,5,0,1/3/2019,0,15
4,"Anderson, Carol",10069,0,2,0,5,5,3,0,50825,1,19,Production Technician I,MA,2169,9/8/1989,F,Divorced,US Citizen,No,White,7/11/2011,9/6/2016,return to school,Voluntarily Terminated,Production,Webster Butler,39.0,Google Search,Fully Meets,5.00,4,0,2/1/2016,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
306,"Woodson, Jason",10135,0,0,1,1,5,3,0,65893,0,20,Production Technician II,MA,1810,5/11/1985,M,Single,US Citizen,No,White,7/7/2014,NaN,N/A-StillEmployed,Active,Production,Kissy Sullivan,20.0,LinkedIn,Fully Meets,4.07,4,0,2/28/2019,0,13
307,"Ybarra, Catherine",10301,0,0,0,5,5,1,0,48513,1,19,Production Technician I,MA,2458,5/4/1982,F,Single,US Citizen,No,Asian,9/2/2008,9/29/2015,Another position,Voluntarily Terminated,Production,Brannon Miller,12.0,Google Search,PIP,3.20,2,0,9/2/2015,5,4
308,"Zamora, Jennifer",10010,0,0,0,1,3,4,0,220450,0,6,CIO,MA,2067,8/30/1979,F,Single,US Citizen,No,White,4/10/2010,NaN,N/A-StillEmployed,Active,IT/IS,Janet King,2.0,Employee Referral,Exceeds,4.60,5,6,2/21/2019,0,16
309,"Zhou, Julia",10043,0,0,0,1,3,3,0,89292,0,9,Data Analyst,MA,2148,2/24/1979,F,Single,US Citizen,No,White,3/30/2015,NaN,N/A-StillEmployed,Active,IT/IS,Simon Roup,4.0,Employee Referral,Fully Meets,5.00,3,5,2/1/2019,0,11


In [10]:
#dropping irrelevant columns
data_cleaned = data.drop(columns = ['EmpID','MarriedID','MaritalStatusID', 'EmpStatusID', 'DeptID', 'PerfScoreID', 'FromDiversityJobFairID', 'ManagerName','ManagerID','State','Position','MaritalDesc','Department','ManagerName','ManagerID','RecruitmentSource','EngagementSurvey','LastPerformanceReview_Date', 'TermReason'])
data_cleaned

,Employee_Name,GenderID,Salary,Termd,PositionID,Zip,DOB,Sex,CitizenDesc,HispanicLatino,RaceDesc,DateofHire,DateofTermination,EmploymentStatus,PerformanceScore,EmpSatisfaction,SpecialProjectsCount,DaysLateLast30,Absences
0,"Adinolfi, Wilson K",1,62506,0,19,1960,7/10/1983,M,US Citizen,No,White,7/5/2011,NaN,Active,Exceeds,5,0,0,1
1,"Ait Sidi, Karthikeyan",1,104437,1,27,2148,5/5/1975,M,US Citizen,No,White,3/30/2015,6/16/2016,Voluntarily Terminated,Fully Meets,3,6,0,17
2,"Akinkuolie, Sarah",0,64955,1,20,1810,9/19/1988,F,US Citizen,No,White,7/5/2011,9/24/2012,Voluntarily Terminated,Fully Meets,3,0,0,3
3,"Alagbe,Trina",0,64991,0,19,1886,9/27/1988,F,US Citizen,No,White,1/7/2008,NaN,Active,Fully Meets,5,0,0,15
4,"Anderson, Carol",0,50825,1,19,2169,9/8/1989,F,US Citizen,No,White,7/11/2011,9/6/2016,Voluntarily Terminated,Fully Meets,4,0,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
306,"Woodson, Jason",1,65893,0,20,1810,5/11/1985,M,US Citizen,No,White,7/7/2014,NaN,Active,Fully Meets,4,0,0,13
307,"Ybarra, Catherine",0,48513,1,19,2458,5/4/1982,F,US Citizen,No,Asian,9/2/2008,9/29/2015,Voluntarily Terminated,PIP,2,0,5,4
308,"Zamora, Jennifer",0,220450,0,6,2067,8/30/1979,F,US Citizen,No,White,4/10/2010,NaN,Active,Exceeds,5,6,0,16
309,"Zhou, Julia",0,89292,0,9,2148,2/24/1979,F,US Citizen,No,White,3/30/2015,NaN,Active,Fully Meets,3,5,0,11


In [11]:
#drop all active employees by dropping all rows with not Date of Termination listed 
print(data_cleaned.isnull().sum())
data_cleaned=data_cleaned.dropna()


Employee_Name             0
GenderID                  0
Salary                    0
Termd                     0
PositionID                0
Zip                       0
DOB                       0
Sex                       0
CitizenDesc               0
HispanicLatino            0
RaceDesc                  0
DateofHire                0
DateofTermination       207
EmploymentStatus          0
PerformanceScore          0
EmpSatisfaction           0
SpecialProjectsCount      0
DaysLateLast30            0
Absences                  0
dtype: int64


In [13]:
data_cleaned

,Employee_Name,GenderID,Salary,Termd,PositionID,Zip,DOB,Sex,CitizenDesc,HispanicLatino,RaceDesc,DateofHire,DateofTermination,EmploymentStatus,PerformanceScore,EmpSatisfaction,SpecialProjectsCount,DaysLateLast30,Absences
1,"Ait Sidi, Karthikeyan",1,104437,1,27,2148,5/5/1975,M,US Citizen,No,White,3/30/2015,6/16/2016,Voluntarily Terminated,Fully Meets,3,6,0,17
2,"Akinkuolie, Sarah",0,64955,1,20,1810,9/19/1988,F,US Citizen,No,White,7/5/2011,9/24/2012,Voluntarily Terminated,Fully Meets,3,0,0,3
4,"Anderson, Carol",0,50825,1,19,2169,9/8/1989,F,US Citizen,No,White,7/11/2011,9/6/2016,Voluntarily Terminated,Fully Meets,4,0,0,2
10,"Baczenski, Rachael",0,54670,1,19,1902,1/12/1974,F,US Citizen,Yes,Black or African American,1/10/2011,1/12/2017,Voluntarily Terminated,Fully Meets,4,0,0,12
11,"Barbara, Thomas",1,47211,1,19,2062,2/21/1974,M,US Citizen,Yes,Black or African American,4/2/2012,9/19/2016,Voluntarily Terminated,Fully Meets,3,0,0,15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
301,"Wilber, Barry",1,55140,1,19,2324,9/9/1965,M,Eligible NonCitizen,No,White,5/16/2011,9/7/2015,Voluntarily Terminated,Fully Meets,3,0,0,7
302,"Wilkes, Annie",0,58062,1,19,1876,7/30/1983,F,US Citizen,No,White,1/10/2011,5/14/2012,Voluntarily Terminated,Fully Meets,5,0,0,9
303,"Williams, Jacquelyn",0,59728,1,19,2109,10/2/1969,F,US Citizen,Yes,Black or African American,1/9/2012,6/27/2015,Voluntarily Terminated,Fully Meets,4,0,0,16
304,"Winthrop, Jordan",1,70507,1,20,2045,11/7/1958,M,US Citizen,No,White,1/7/2013,2/21/2016,Voluntarily Terminated,Exceeds,3,0,0,7


In [16]:
data_cleaned.describe()

,GenderID,Salary,Termd,PositionID,Zip,EmpSatisfaction,SpecialProjectsCount,DaysLateLast30,Absences
count,88.000000,88.000000,88.0,88.000000,88.000000,88.000000,88.000000,88.000000,88.000000
mean,0.420455,63949.204545,1.0,18.738636,2946.113636,3.897727,0.511364,0.409091,10.954545
std,0.496461,17012.771985,0.0,3.881860,4625.629077,0.858223,1.568324,1.318623,5.795181
min,0.000000,45115.000000,1.0,2.000000,1420.000000,2.000000,0.000000,0.000000,1.000000
25%,0.000000,53414.000000,1.0,19.000000,1901.750000,3.000000,0.000000,0.000000,6.000000
50%,0.000000,60512.000000,1.0,19.000000,2140.500000,4.000000,0.000000,0.000000,11.000000
75%,1.000000,68529.750000,1.0,20.000000,2374.500000,5.000000,0.000000,0.000000,16.000000
max,1.000000,148999.000000,1.0,29.000000,37129.000000,5.000000,7.000000,6.000000,20.000000
